In [28]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import *

### Read in Source Data and Create 3 Scenarios

In [181]:
df = pd.read_csv("TSPTW_150.txt")

In [182]:
all_vals = []
for i in range(df.shape[0]):
    vals = [val for val in df.iloc[i,0].strip().split(" ") if val != ""]
    all_vals.append(vals)

df_data = pd.DataFrame(all_vals, columns = ["CUST NO.", "X", "Y", "DEMAND", "READY TIME", "DUE DATE", "SERVICE TIME"])
df_data = df_data.iloc[:-1,:]
df_data["COORDINATES"] = df_data.apply(lambda row: (row["X"], row["Y"]), axis=1)
df_data["DEMAND"] = np.random.randint(1,5,df_data.shape[0])
df_data["SERVICE TIME"] = np.random.randint(1,9,df_data.shape[0])
df_data = df_data.drop(columns=["X", "Y"])

In [193]:
df_small = df_data.iloc[1:].sample(n = 80, replace = False, random_state=42).copy()
df_medium = df_data.iloc[1:].sample(n = 100, replace = False, random_state=42).copy()
df_large = df_data.iloc[1:].sample(n = 120, replace = False, random_state=42).copy()

In [194]:
initial = pd.DataFrame([df_data.iloc[0].to_list()], columns = df_data.columns)

In [195]:
df_small = pd.concat([initial, df_small]).reset_index(drop=True)
df_medium = pd.concat([initial, df_medium]).reset_index(drop=True)
df_large = pd.concat([initial, df_large]).reset_index(drop=True)

### Create Distance and Time Matrices

In [246]:
import math
# Compute pairwise times matrix

def distance(df, col, row1, row2):
    c1 = df.loc[row1, col]
    c2 = df.loc[row2, col]
    diff = (float(c1[0])-float(c2[0]), float(c1[1])-float(c2[1]))
    return math.sqrt(diff[0]*diff[0]+diff[1]*diff[1])

small_dist = {(c1, c2): distance(df_small, "COORDINATES", c1, c2) for c1 in range(df_small.shape[0]) for c2 in range(df_small.shape[0])}
medium_dist = {(c1, c2): distance(df_medium, "COORDINATES", c1, c2) for c1 in range(df_medium.shape[0]) for c2 in range(df_medium.shape[0])}
large_dist = {(c1, c2): distance(df_large, "COORDINATES", c1, c2) for c1 in range(df_large.shape[0]) for c2 in range(df_large.shape[0])}

In [247]:
small_dist = []
for row1 in range(df_small.shape[0]):
    row_vals = []
    for row2 in range(df_small.shape[0]):
        row_vals.append(distance(df_small, "COORDINATES", row1, row2))
    small_dist.append(row_vals)

In [248]:
medium_dist = []
for row1 in range(df_medium.shape[0]):
    row_vals = []
    for row2 in range(df_medium.shape[0]):
        row_vals.append(distance(df_medium, "COORDINATES", row1, row2))
    medium_dist.append(row_vals)

In [249]:
large_dist = []
for row1 in range(df_large.shape[0]):
    row_vals = []
    for row2 in range(df_large.shape[0]):
        row_vals.append(distance(df_large, "COORDINATES", row1, row2))
    large_dist.append(row_vals)

In [250]:
dist_small = np.asarray(small_dist)
dist_medium = np.asarray(medium_dist)
dist_large = np.asarray(large_dist)

### Model Implementation - Small

In [251]:
I = df_small.shape[0]

6561

In [ ]:
prob = gp.Model()

# Variables: is city 'i' adjacent to city 'j' on the tour?
vars = prob.addVars(small_dist.keys(), obj=small_dist, vtype=GRB.BINARY, name='x')

# Symmetric direction: Copy the object
for i, j in vars.keys():
    vars[j, i] = vars[i, j]  # edge in opposite direction

# Only one edge to each venue
cons = prob.addConstrs(vars.sum(i, '*') == 1 for i in I)

In [252]:
I = len(dist_small)
J = len(dist_small[0])

# Start Times
A = df_small["READY TIME"].astype(float).to_numpy()

# End Times
B = df_small["DUE DATE"].astype(float).to_numpy()

# distance matrix
D = dist_small

In [253]:
prob = gp.Model("TSPTW - Concerts - Small")

X = prob.addVars(I, J, vtype=GRB.BINARY, name=[f"({i}), ({j})" for i in range(I) for j in range(J)])
# U = prob.addVars(I, lb=1, ub=I-1, vtype=GRB.INTEGER, name=[str(i) + "dum" for i in range(I)])
TI = prob.addVars(I, vtype=GRB.INTEGER, name=[f"TI_{i}" for i in range(I)])
TJ = prob.addVars(J, vtype=GRB.INTEGER, name=[f"TJ_{j}" for j in range(J)])

prob.setObjective(gp.quicksum(X[i,j]*D[i][j] for i in range(I) for j in range(J)), GRB.MINIMIZE)

In [254]:
for i in range(I):
    # Initiation of Arrival to first node
    prob.addConstr(TI[i] - D[0][i]*X[0,i] >= 0)

    # start of time window
    prob.addConstr(TI[i] >= A[i])

    # end of time window
    prob.addConstr(TJ[i] <= B[i])

    # only one per row
    prob.addConstr(gp.quicksum(X[i,j] for j in range(J)) == 1, name = "row" + str(i))

    # cannot go along diagonals
    prob.addConstr(X[i,i] == 0)

for j in range(J):
    # only one per column
    prob.addConstr(gp.quicksum(X[i,j] for i in range(I)) == 1, name = "col" + str(j))

In [255]:
# Sub tour Elimination
for i in range(I):
    for j in range(J):
        if i == j:
            continue
        else:
            prob.addConstr(TI[i] - TJ[j] + (B[i] - A[j] + D[i][j])*X[i,j] <= B[i] - A[j], name = "row" + str(i) + " - col" + str(j))

In [256]:
prob.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6966 rows, 6723 columns and 32966 nonzeros
Model fingerprint: 0x1b1213af
Variable types: 0 continuous, 6723 integer (6561 binary)
Coefficient statistics:
  Matrix range     [4e-02, 1e+03]
  Objective range  [1e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 6815 rows and 3764 columns
Presolve time: 0.12s
Presolved: 151 rows, 2959 columns, 5925 nonzeros
Variable types: 0 continuous, 2959 integer (2959 binary)

Root relaxation: objective 5.453112e+02, 161 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     545.3111592  545.31116

In [230]:
print('Sensitivity Analysis (SA)\n ObjVal =', prob.ObjVal)
prob.printAttr(['X', 'Obj'])

Sensitivity Analysis (SA)
 ObjVal = 261.02742367748505

    Variable            X          Obj 
--------------------------------------
    (0), (1)           -0      42.2019 
    (0), (2)           -0      18.9737 
    (0), (3)           -0           10 
    (0), (4)           -0      37.9473 
    (0), (5)           -0      30.5287 
    (0), (6)           -0      27.4591 
    (0), (7)           -0      30.0832 
    (0), (8)           -0      37.0135 
    (0), (9)           -0       28.178 
   (0), (10)           -0      28.4605 
   (0), (11)           -0      23.3238 
   (0), (12)           -0      15.6525 
   (0), (13)           -0      26.2488 
   (0), (14)           -0      42.5793 
   (0), (15)           -0      29.1548 
   (0), (16)           -0      13.6015 
   (0), (17)           -0      33.7343 
   (0), (18)           -0      23.4094 
   (0), (19)           -0      24.0832 
   (0), (20)           -0      5.38516 
   (0), (21)           -0      47.5079 
   (0), (22)           -0

In [257]:
tot = 0
for v in prob.getVars():
    if v.x > 0:
        print(v.varName, v.x, v.Obj)
        tot += v.Obj

(0), (52) 1.0 3.605551275463989
(1), (38) 1.0 1.4142135623730951
(2), (66) 1.0 2.8284271247461903
(3), (60) 1.0 13.892443989449804
(4), (71) 1.0 13.038404810405298
(5), (55) 1.0 3.605551275463989
(6), (0) 1.0 27.459060435491963
(7), (72) 1.0 4.47213595499958
(8), (77) 1.0 4.242640687119285
(9), (5) 1.0 3.1622776601683795
(10), (7) 1.0 2.23606797749979
(11), (40) 1.0 4.242640687119285
(12), (69) 1.0 21.93171219946131
(13), (41) 1.0 23.194827009486403
(14), (1) 1.0 3.1622776601683795
(15), (59) 1.0 2.8284271247461903
(16), (27) 1.0 8.602325267042627
(17), (31) 1.0 1.4142135623730951
(18), (61) 1.0 3.605551275463989
(19), (46) 1.0 1.0
(20), (58) 1.0 12.529964086141668
(21), (62) 1.0 1.4142135623730951
(22), (16) 1.0 7.615773105863909
(23), (39) 1.0 3.605551275463989
(24), (65) 1.0 5.0
(25), (26) 1.0 1.4142135623730951
(26), (25) 1.0 1.4142135623730951
(27), (22) 1.0 4.47213595499958
(28), (11) 1.0 7.211102550927978
(29), (12) 1.0 6.082762530298219
(30), (45) 1.0 5.656854249492381
(31), (5

### Model Implementation - Medium

In [258]:
I = len(dist_medium)
J = len(dist_medium[0])

# Start Times
A = df_medium["READY TIME"].astype(float).to_numpy()

# End Times
B = df_medium["DUE DATE"].astype(float).to_numpy()

# distance matrix
D = dist_medium

In [259]:
prob = gp.Model("TSPTW - Concerts - Medium")

X = prob.addVars(I, J, vtype=GRB.BINARY, name=[f"({i}), ({j})" for i in range(I) for j in range(J)])
# U = prob.addVars(I, lb=1, ub=I-1, vtype=GRB.INTEGER, name=[str(i) + "dum" for i in range(I)])
TI = prob.addVars(I, vtype=GRB.INTEGER, name=[f"TI_{i}" for i in range(I)])
TJ = prob.addVars(J, vtype=GRB.INTEGER, name=[f"TJ_{j}" for j in range(J)])

prob.setObjective(gp.quicksum(X[i,j]*D[i][j] for i in range(I) for j in range(J)), GRB.MINIMIZE)

In [260]:
for i in range(I):
    # Initiation of Arrival to first node
    prob.addConstr(TI[i] - D[0][i]*X[0,i] >= 0)

    # start of time window
    prob.addConstr(TI[i] >= A[i])

    # end of time window
    prob.addConstr(TJ[i] <= B[i])

    # only one per row
    prob.addConstr(gp.quicksum(X[i,j] for j in range(J)) == 1, name = "row" + str(i))

    # cannot go along diagonals
    prob.addConstr(X[i,i] == 0)

for j in range(J):
    # only one per column
    prob.addConstr(gp.quicksum(X[i,j] for i in range(I)) == 1, name = "col" + str(j))

In [261]:
# Sub tour Elimination
for i in range(I):
    for j in range(J):
        if i == j:
            continue
        else:
            prob.addConstr(TI[i] - TJ[j] + (B[i] - A[j] + D[i][j])*X[i,j] <= B[i] - A[j], name = "row" + str(i) + " - col" + str(j))

In [262]:
prob.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 10706 rows, 10403 columns and 51206 nonzeros
Model fingerprint: 0xd2182b20
Variable types: 0 continuous, 10403 integer (10201 binary)
Coefficient statistics:
  Matrix range     [4e-02, 1e+03]
  Objective range  [1e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 10519 rows and 5876 columns
Presolve time: 0.11s
Presolved: 187 rows, 4527 columns, 9061 nonzeros
Variable types: 0 continuous, 4527 integer (4527 binary)

Root relaxation: objective 5.997767e+02, 186 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     599.7766650  599.

In [263]:
tot = 0
for v in prob.getVars():
    if v.x > 0:
        print(v.varName, v.x, v.Obj)
        tot += v.Obj

(0), (52) 1.0 3.605551275463989
(1), (14) 1.0 3.1622776601683795
(2), (66) 1.0 2.8284271247461903
(3), (26) 1.0 8.94427190999916
(4), (50) 1.0 5.830951894845301
(5), (55) 1.0 3.605551275463989
(6), (0) 1.0 27.459060435491963
(7), (87) 1.0 1.0
(8), (77) 1.0 4.242640687119285
(9), (93) 1.0 0.0
(10), (72) 1.0 3.0
(11), (40) 1.0 4.242640687119285
(12), (85) 1.0 8.54400374531753
(13), (41) 1.0 23.194827009486403
(14), (38) 1.0 2.8284271247461903
(15), (83) 1.0 3.1622776601683795
(16), (82) 1.0 6.082762530298219
(17), (7) 1.0 4.123105625617661
(18), (61) 1.0 3.605551275463989
(19), (46) 1.0 1.0
(20), (58) 1.0 12.529964086141668
(21), (62) 1.0 1.4142135623730951
(22), (27) 1.0 4.47213595499958
(23), (39) 1.0 3.605551275463989
(24), (88) 1.0 4.47213595499958
(25), (94) 1.0 4.242640687119285
(26), (25) 1.0 1.4142135623730951
(27), (97) 1.0 4.47213595499958
(28), (11) 1.0 7.211102550927978
(29), (98) 1.0 3.1622776601683795
(30), (45) 1.0 5.656854249492381
(31), (57) 1.0 3.1622776601683795
(32), 